# TC4 - Lab exercises 2

In this notebook, we are going to improve the POS tagger of last week. Instead of using a naive Bayes classifier, we will rely on a HMM where:
- the hidden states are POS tags
- the observations are words

It is a first order HMM where probabilities are defined as follows:
$$
p(y_1...y_n, x_1...x_n) = p(y_1) \prod_{i=2}^n p(y_i | y_{i-1}) \prod_{i=1}^n p(x_i | y_i)
$$

In [111]:
import nltk
import numpy as np
import sys
import math

# 1. Data

We first need to load and split the data between train and test data. You need to report the code from last week with the same split (90% train / 10% test).

In [112]:
# import dataset
data = list(nltk.corpus.brown.tagged_sents(tagset='universal'))

In [113]:
train_data = []
test_data = []

import random
random.shuffle(data) # We start by shuffling the dataset to break the correlation between successive sentences
test_data = data[:len(data)//10]
train_data = data[len(data)//10:]

In [114]:
len(train_data), len(test_data)

(51606, 5734)

We will store the parameters of the HMM in numpy arrays. Therefore, to simplify the model, we rely on a dictionnary that maps words to tokens:
- the constructor takes as argument an iteratable over strings (e.g. list of strings) containing the vocabulary to store in the dictionnary
- you can set unk="\*UNK\*" to add entry for unknown strings (do not do it for POS tags!)
- len(dict) gives you the numbers of entry in the dict
- str_to_id maps a string to an index
- id_to_str gives you the string stored at a given index

In [115]:
class Dict:
    def __init__(self, words, unk=None):
        self._unk = unk
        self._word_to_id = dict()
        self._id_to_word = list()

        if unk in words:
            raise RuntimeError("UNK word exists in vocabulary")

        if unk is not None:
            self.unk_index = self._add_word(unk)

        for word in words:
            self._add_word(word)

    # for internal use only!
    def _add_word(self, word):
        if word not in self._word_to_id:
            id = len(self._id_to_word)
            self._word_to_id[word] = id
            self._id_to_word.append(word)
            return id
        else:
            return self._word_to_id[word]

    def str_to_id(self, word):
        if self._unk is not None:
            return self._word_to_id.get(word, self.unk_index)
        else:
            return self._word_to_id[word]

    def id_to_str(self, id):
        return self._id_to_word[id]

    def __len__(self):
        return len(self._word_to_id)

    def has_unk(self):
        return self._unk is not None
    
    def unk(self):
        return self.unk_index

Example:

In [116]:
test_dict = Dict(["a", "b", "c"], unk="*UNK*")
print("N. entry: ", len(test_dict))
print("Index of \"b\":", test_dict.str_to_id("b"))
# the following line does not throw an error because we gave a unk word
print("Index of \"e\":", test_dict.str_to_id("e"))

N. entry:  4
Index of "b": 2
Index of "e": 0


We now build the dictionnary of words and tags. We will restrict the dictionnary of words to contain only words that appears 10 or more times in the training data (use the code of last time).

For the dictionnary of words, the the unk parameters to any string you want. For the dictionnary of POS tags, do not set an unk word!

In [117]:
def compute_word_counts(data):
    distribution = {}
    for sentence in data:
        for word, tag in sentence:
            if word in distribution:
                distribution[word] += 1
            else:
                distribution[word] = 1
    return distribution
word_counts = compute_word_counts(train_data)
word_dict = Dict([word for word in word_counts if word_counts[word] >= 10], unk="*UNK*")
tags = []
for sentence in train_data:
    for word, tag in sentence:
        tags.append(tag)
tags_dict = Dict(tags)

# 2. Hidden Markov Model

The HMM class is a simple container for:
- the dictionnary of hidden states y_dict (i.e. dictionnary of tags)
- the dictionnary of observations x_dict (i.e. dictionnary of words)
- the parameters of the HMM:
    * init_prob $\in \mathbb R^{|Y|}$: initial tag probabilities $p(y_0) = init\_prob[y_0]$
    * transition_prob $\in \mathbb R^{|Y| \times |Y|}$: tag transition probabilities $p(y_i | y_{i - 1}) = transition\_prob[y_{i - 1}, y_i]$
    * observation_prob $\in \mathbb R^{|Y| \times |X|}$: observation probabilities $p(x_i | y_i) = observation\_prob[y_i, x_i]$

In [118]:
class HMM:
    def __init__(self, y_dict, x_dict):
        if not isinstance(y_dict, Dict) or not isinstance(x_dict, Dict):
            raise RuntimeError("Arguments must be of type Dict")

        self.y_dict = y_dict
        self.x_dict = x_dict
        n_y = len(y_dict)
        n_x = len(x_dict)
        
        self.init_prob = np.zeros((n_y,), float) 
        self.transition_prob = np.zeros((n_y, n_y), float) 
        self.observation_prob = np.zeros((n_y, n_x), float) 

## 2.1 Learning

Compute the matrices of probabilities hmm.init_prob, hmm.observation_prob and hmm.transition_prob from the data.

You **must** smooth the distributions!

In [119]:
n_x = len(word_dict)
n_y = len(tags_dict)

hmm = HMM(tags_dict, word_dict)
for sentence in train_data:
    hmm.init_prob[tags_dict.str_to_id(sentence[0][1])] += 1.
hmm.init_prob += np.full(n_y, 1.)
hmm.init_prob /= np.sum(hmm.init_prob)

for sentence in train_data:
    for word, tag in sentence:
        hmm.observation_prob[tags_dict.str_to_id(tag), word_dict.str_to_id(word)] += 1.
hmm.observation_prob += np.full((n_y, n_x), 1.)
for y in range(n_y):
    hmm.observation_prob[y] /= np.sum(hmm.observation_prob[y])

for sentence in train_data:
    for i in range(1, len(sentence)):
        hmm.transition_prob[tags_dict.str_to_id(sentence[i-1][1]), tags_dict.str_to_id(sentence[i][1])] += 1.
hmm.transition_prob += np.full((n_y, n_y), 1.)
for y in range(n_y):
    hmm.transition_prob[y] /= np.sum(hmm.transition_prob[y])

The following three cells check that the distribution you computed correctly sum to one. The first cell should output 1.0, the two others should output array containing twelve times the number 1.

In [120]:
hmm.init_prob.sum()

0.9999999999999999

In [121]:
hmm.observation_prob.sum(1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [122]:
hmm.transition_prob.sum(1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

# 2.2. Viterbi

Implement the viterbi **without** computing in the log domain. What tagging accuracy do you achieve? How is it compared to the naive Bayes model of last week?

In [123]:
def viterbi(hmm, words):
    """
    Input:
    - hmm: an HMM object
    - words: a list of words (ie a sentence)
    Return:
    - a list of POS tags
    """
    x = [hmm.x_dict.str_to_id(word) for word in words]
    n = len(words)
    chart = np.zeros((n_y, n))
    backptr = np.zeros((n_y, n), dtype='int')
    for y in range(n_y):
        chart[y, 0] = hmm.init_prob[y] * hmm.observation_prob[y, x[0]]
    for j in range(1, n):
        for y in range(n_y):
            ypmax, maxi = -1, -np.inf
            for yp in range(n_y):
                val = hmm.transition_prob[yp, y] * hmm.observation_prob[y, x[j]] * chart[yp, j-1]
                if val > maxi:
                    ypmax = yp
                    maxi = val
            chart[y, j] = maxi
            backptr[y, j] = ypmax
    pred = [-1] * n
    pred[n-1] = np.argmax(chart[:,n-1])
    for j in reversed(range(0, n-1)):
        pred[j] = backptr[pred[j+1], j+1]
    for i in range(n):
        pred[i] = hmm.y_dict.id_to_str(pred[i])
    return pred

In [124]:
# Evaluate the HMM using the viterbi
n_tags = 0
n_correct_tags = 0
for sentence in test_data:
    words = [w for w, t in sentence]
    pred = viterbi(hmm, words)
    n_tags += len(sentence)
    n_correct_tags += sum(1 for w in range(len(sentence))  if sentence[w][1] == pred[w])
print("Tagging accuracy: %.2f" % (100 * n_correct_tags / n_tags))

Tagging accuracy: 93.22


# 2.3. Viterbi in the log domain

Copy/paste you code from the previous cell and change it to compute in the log domain. What tagging accuracy do you achieve? How is it compared to the naive Bayes model of last week and to the previous implementation of the viterbi?

In [125]:
def viterbi_log(hmm, words):
    """
    Input:
    - hmm: an HMM object
    - words: a list of words (ie a sentence)
    Return:
    - a list of POS tags
    """
    x = [hmm.x_dict.str_to_id(word) for word in words]
    n = len(words)
    chart = np.zeros((n_y, n))
    backptr = np.zeros((n_y, n), dtype='int')
    init_prob = np.log(hmm.init_prob)
    observation_prob = np.log(hmm.observation_prob)
    transition_prob = np.log(hmm.transition_prob)
    
    for y in range(n_y):
        chart[y, 0] = init_prob[y] + observation_prob[y, x[0]]
    for j in range(1, n):
        for y in range(n_y):
            ypmax, maxi = -1, -np.inf
            for yp in range(n_y):
                val = transition_prob[yp, y] + observation_prob[y, x[j]] + chart[yp, j-1]
                if val > maxi:
                    ypmax = yp
                    maxi = val
            chart[y, j] = maxi
            backptr[y, j] = ypmax
    pred = [-1] * n
    pred[n-1] = np.argmax(chart[:,n-1])
    for j in reversed(range(0, n-1)):
        pred[j] = backptr[pred[j+1], j+1]
    for i in range(n):
        pred[i] = hmm.y_dict.id_to_str(pred[i])
    return pred

In [126]:
# Evaluate the HMM using the viterbi in the log domain

n_tags = 0
n_correct_tags = 0
for sentence in test_data:
    words = [w for w, t in sentence]
    pred = viterbi_log(hmm, words)
    n_tags += len(sentence)
    n_correct_tags += sum(1 for w in range(len(sentence))  if sentence[w][1] == pred[w])

print("Tagging accuracy: %.2f" % (100 * n_correct_tags / n_tags))

Tagging accuracy: 93.24


# 3. Marginalization

As a last exercise, implement function that evaluate the probability of a sequence of words and a sequence of hidden states given a HMM.

In [127]:
def probabilit_y(hmm, tags):
    prob = hmm.init_prob[tags_dict.str_to_id(tags[0])]
    for i in range(1, len(tags)):
        prob *= hmm.transition_prob[tags_dict.str_to_id(tags[i-1]), tags_dict.str_to_id(tags[i])]
    return prob

In [128]:
tags = "DET NOUN VERB DET ADJ NOUN .".split()
print(probabilit_y(hmm, tags))
random.shuffle(tags)
print(probabilit_y(hmm, tags))

0.0001539949469698149
1.7378512070604983e-08


In [129]:
def probabilit_x(hmm, words):
    n = len(words)
    x = [word_dict.str_to_id(word) for word in words]
    chart = np.zeros((n_y, n))
    for y in range(n_y):
        chart[y, 0] = hmm.init_prob[y] * hmm.observation_prob[y, x[0]]
    for j in range(1, n):
        for y in range(n_y):
            chart[y, j] = sum(hmm.transition_prob[yp, y] * hmm.observation_prob[y, x[j]] * chart[yp, j-1] for yp in range(n_y))
    return sum(chart[y, n-1] for y in range(n_y))

In [130]:
sentence = "This is a sentence .".split()
print(probabilit_x(hmm, sentence))
random.shuffle(sentence)
print(probabilit_x(hmm, sentence))

1.0083557975588482e-12
2.046577199314669e-15
